In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import tensorflow as tf
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel

print("Torch CUDA:",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
print("GPU Available:", tf.config.list_physical_devices('GPU'))

2024-04-11 10:16:52.256444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 10:16:52.256507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 10:16:52.285351: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 10:16:52.347191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 10:16:53.375556: W tensorflow/compiler/tf2

Torch CUDA: True
Using device: cuda
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-11 10:16:54.622198: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 10:16:54.628054: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 10:16:54.628658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# load the dataset -- motes.csv
notes = pd.read_csv('../data/notes.csv')

# check the dimension of the dataset
num_row, num_col = notes.shape
print('The dataset contains {} rows and {} columns.'.format(num_row, num_col))

# check the number of patients in the dataset
print('Number of patients: ', notes.id.nunique())

# view the first 5 rows of the dataset
display(notes.head())

# summary of the dataset
notes.info()

The dataset contains 81206 rows and 6 columns.
Number of patients:  20414


,note_id,id,note_type,note_seq,charttime,text
0,10002930-RR-19,25696644,RR,19,4/14/96 9:57,HISTORY: Altered mental status.\n\nTECHNIQUE:...
1,10003046-RR-7,26048429,RR,7,1/2/54 16:29,"INDICATION: Esophageal carcinoma, status post..."
2,10003046-RR-8,26048429,RR,8,1/3/54 7:33,HISTORY: Postop day one interval change.\n\nC...
3,10003400-RR-65,20214994,RR,65,2/24/37 16:01,EXAMINATION: CHEST (PRE-OP PA AND LAT)\n\nIND...
4,10003400-RR-66,20214994,RR,66,2/25/37 19:23,EXAMINATION: ABDOMINAL RADIOGRAPHS\n\nINDICAT...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81206 entries, 0 to 81205
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   note_id    81206 non-null  object
 1   id         81206 non-null  int64 
 2   note_type  81206 non-null  object
 3   note_seq   81206 non-null  int64 
 4   charttime  81206 non-null  object
 5   text       81206 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.7+ MB


In [4]:
# load the dataset -- static.csv
static = pd.read_csv('../data/static.csv')

# check the dimension of the dataset
num_row, num_col = static.shape
print('The dataset contains {} rows and {} columns.'.format(num_row, num_col))

# check the number of patients in the dataset
print('Number of patients: ', static.id.nunique())

# view the first 5 rows of the dataset
pd.set_option('display.max_columns', num_col)
display(static.head())

# summary of the dataset
static.info()

The dataset contains 20414 rows and 25 columns.
Number of patients:  20414


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,weight_admit,height,admission_type,first_careunit,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,M,OTHER,58.441631,92.0,170.0,ELECTIVE,Trauma SICU (TSICU),11,0,0,0,0,0,1,1,1,0,0
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,F,WHITE,79.748598,55.0,NaN,EW EMER.,Trauma SICU (TSICU),5,0,0,0,0,0,1,0,1,1,0
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,F,WHITE,81.717397,59.0,NaN,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),8,0,0,1,1,0,0,0,0,1,0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,F,BLACK/AFRICAN AMERICAN,63.769278,73.6,160.0,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),8,0,0,1,1,0,1,1,1,1,0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,F,WHITE,84.191979,79.1,160.0,OBSERVATION ADMIT,Medical/Surgical Intensive Care Unit (MICU/SICU),6,1,0,1,1,0,0,1,1,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20414 entries, 0 to 20413
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   20414 non-null  int64  
 1   hosp_admittime       20414 non-null  object 
 2   hosp_dischtime       20414 non-null  object 
 3   icu_intime           20414 non-null  object 
 4   icu_outtime          20414 non-null  object 
 5   los_icu              20414 non-null  float64
 6   icu_death            20414 non-null  int64  
 7   gender               20414 non-null  object 
 8   race                 20414 non-null  object 
 9   admission_age        20414 non-null  float64
 10  weight_admit         19802 non-null  float64
 11  height               11000 non-null  float64
 12  admission_type       20414 non-null  object 
 13  first_careunit       20414 non-null  object 
 14  charlson_score       20414 non-null  int64  
 15  atrial_fibrillation  20414 non-null 

In [5]:
static_sub = static[['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','los_icu','icu_death']]
notes_sub = notes[['id','charttime','text']]

# merge the static and notes dataframe
merged_df = pd.merge(static_sub, notes_sub, on='id', how='inner')

# check the dimension of the merged dataframe
num_row, num_col = merged_df.shape
print('The merged dataframe contains {} rows and {} columns.'.format(num_row, num_col))

# check the number of patients after merging 2 dataframes
print('Number of patients in the merged dataframe: ', merged_df.id.nunique())

# view the first 5 rows of the merged dataframe
display(merged_df.head())

# summary of the merged dataframe
merged_df.info()

The merged dataframe contains 81206 rows and 9 columns.
Number of patients in the merged dataframe:  20414


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,charttime,text
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,6/11/11 16:13,"INDICATION: Esophageal cancer, status post re..."
1,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,6/12/11 6:10,PORTABLE CHEST ___\n\nCOMPARISON: ___ radiogr...
2,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,9/30/28 13:18,EXAMINATION: CT ABDOMEN AND PELVIS\n\nINDICAT...
3,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,10/1/28 5:28,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...
4,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,9/20/41 1:01,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81206 entries, 0 to 81205
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              81206 non-null  int64  
 1   hosp_admittime  81206 non-null  object 
 2   hosp_dischtime  81206 non-null  object 
 3   icu_intime      81206 non-null  object 
 4   icu_outtime     81206 non-null  object 
 5   los_icu         81206 non-null  float64
 6   icu_death       81206 non-null  int64  
 7   charttime       81206 non-null  object 
 8   text            81206 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 5.6+ MB


In [6]:
# convert the data type for all date columns into datetime with date parsing
merged_df['hosp_admittime'] = pd.to_datetime(merged_df['hosp_admittime'], format='%m/%d/%y %H:%M')
merged_df['hosp_dischtime'] = pd.to_datetime(merged_df['hosp_dischtime'], format='%m/%d/%y %H:%M')
merged_df['icu_intime'] = pd.to_datetime(merged_df['icu_intime'], format='%m/%d/%y %H:%M')
merged_df['icu_outtime'] = pd.to_datetime(merged_df['icu_outtime'], format='%m/%d/%y %H:%M')
merged_df['charttime'] = pd.to_datetime(merged_df['charttime'], format='%m/%d/%y %H:%M')

In [7]:
# remove inconsistent dates
merged_df['diff_hosptime'] = merged_df['hosp_dischtime'] - merged_df['hosp_admittime']
merged_df['diff_icutime'] = merged_df['icu_outtime'] - merged_df['icu_intime']

neg_hosp_time = merged_df['diff_hosptime'] < pd.Timedelta(0)
neg_icu_time = merged_df['diff_icutime'] < pd.Timedelta(0)
neg_time = (neg_hosp_time | neg_icu_time)

print(f'Number of rows with inconsistent calculated LOS: {neg_time.sum()}')
print('===Data with negative calculated LOS for hospitalisation or negative calculated LOS for ICU===')
display(merged_df[neg_time])

cleaned_df1 = merged_df[~neg_time].copy()
print(f'Dimensions of cleaned_df1 after dropping rows with inconsistent dates: {cleaned_df1.shape}')
print('Number of patients in the cleaned_df1: ', cleaned_df1.id.nunique())

Number of rows with inconsistent calculated LOS: 32
===Data with negative calculated LOS for hospitalisation or negative calculated LOS for ICU===


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,charttime,text,diff_hosptime,diff_icutime
5375,25025990,2068-12-27 00:00:00,1969-01-04 14:12:00,2068-12-27 20:12:00,2068-12-31 14:52:00,3.78,0,2068-12-27 17:06:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,-36517 days +14:12:00,3 days 18:40:00
13381,23396294,1994-08-07 00:49:00,1994-08-07 00:00:00,1994-08-07 02:25:00,1994-08-08 04:21:00,1.08,1,1994-08-07 01:06:00,INDICATION: Single episode with head strike.\...,-1 days +23:11:00,1 days 01:56:00
13382,23396294,1994-08-07 00:49:00,1994-08-07 00:00:00,1994-08-07 02:25:00,1994-08-08 04:21:00,1.08,1,1994-08-07 01:06:00,INDICATION: Fall with head strike. Evaluatio...,-1 days +23:11:00,1 days 01:56:00
13383,23396294,1994-08-07 00:49:00,1994-08-07 00:00:00,1994-08-07 02:25:00,1994-08-08 04:21:00,1.08,1,1994-08-07 04:04:00,"AP CHEST, 4:28 A.M., ___\n\nHISTORY: Evaluate...",-1 days +23:11:00,1 days 01:56:00
13384,23396294,1994-08-07 00:49:00,1994-08-07 00:00:00,1994-08-07 02:25:00,1994-08-08 04:21:00,1.08,1,1994-08-07 10:05:00,CHEST ON ___\n\nHISTORY: Left IJ line.\n\nFIN...,-1 days +23:11:00,1 days 01:56:00
19017,28788162,2068-12-27 07:58:00,1969-01-25 12:20:00,2068-12-27 09:45:00,2068-12-31 13:23:00,4.15,0,2068-12-27 00:21:00,EXAMINATION: CT CHEST W/CONTRAST\n\nINDICATIO...,-36496 days +04:22:00,4 days 03:38:00
26584,26539098,2068-12-21 18:36:00,1969-01-09 15:45:00,2068-12-22 16:09:00,2068-12-24 21:18:00,2.21,0,2068-12-21 14:04:00,HISTORY: A ___ female with headache and fever...,-36507 days +21:09:00,2 days 05:09:00
26585,26539098,2068-12-21 18:36:00,1969-01-09 15:45:00,2068-12-22 16:09:00,2068-12-24 21:18:00,2.21,0,2068-12-21 18:04:00,INDICATION: Encephalitis. Evaluate for infec...,-36507 days +21:09:00,2 days 05:09:00
26586,26539098,2068-12-21 18:36:00,1969-01-09 15:45:00,2068-12-22 16:09:00,2068-12-24 21:18:00,2.21,0,2068-12-22 10:42:00,INDICATION: ___ female with new right PICC li...,-36507 days +21:09:00,2 days 05:09:00
26587,26539098,2068-12-21 18:36:00,1969-01-09 15:45:00,2068-12-22 16:09:00,2068-12-24 21:18:00,2.21,0,2068-12-22 15:18:00,INDICATION: ___ female with multiple intracra...,-36507 days +21:09:00,2 days 05:09:00


Dimensions of cleaned_df1 after dropping rows with inconsistent dates: (81174, 11)
Number of patients in the cleaned_df1:  20403


In [8]:
# remove rows with 'charttime' longer than 24h or after hosp_dischtime or after icu_outtime

cleaned_df1['diff_charttime_icuintime'] = cleaned_df1['charttime'] - cleaned_df1['icu_intime']
cleaned_df1['diff_charttime_icuoutime'] = cleaned_df1['charttime'] - cleaned_df1['icu_outtime']
cleaned_df1['diff_charttime_hospadmittime'] = cleaned_df1['charttime'] - cleaned_df1['hosp_admittime']
cleaned_df1['diff_charttime_hospdischtime'] = cleaned_df1['charttime'] - cleaned_df1['hosp_dischtime']

charttime_longer_than_24h = cleaned_df1['diff_charttime_icuintime'] > pd.Timedelta(1, unit='days')
charttime_after_icu_outtime = cleaned_df1['diff_charttime_icuoutime'] > pd.Timedelta(0) 
charttime_after_hosp_dischtime = cleaned_df1['diff_charttime_hospdischtime'] > pd.Timedelta(0) 
charttime_out_of_period = (charttime_longer_than_24h | charttime_after_icu_outtime | charttime_after_hosp_dischtime)
cleaned_df2 = cleaned_df1[~charttime_out_of_period].copy()

print(f'Number of rows with charttime longer than 24h: {charttime_longer_than_24h.sum()}')
print(f'Number of rows with charttime after icu_outtime: {charttime_after_icu_outtime.sum()}')
print(f'Number of rows with charttime after hosp_dischtime: {charttime_after_hosp_dischtime.sum()}')
print(f'Number of rows with charttime out of period of interest: {charttime_out_of_period.sum()}')
print(f'Dimensions of df after dropping rows with charttime out of period of interest: {cleaned_df2.shape}')

Number of rows with charttime longer than 24h: 1
Number of rows with charttime after icu_outtime: 0
Number of rows with charttime after hosp_dischtime: 31
Number of rows with charttime out of period of interest: 32
Dimensions of df after dropping rows with charttime out of period of interest: (81142, 15)


In [9]:
# check the dimension of the dataframe after excluding these records
print('Dimension of cleaned_df: ', cleaned_df2.shape)

# check the number of patients remains in the cleaned_df
print('Number of patients:', cleaned_df2.id.nunique())

# view the first 5 rows of the cleaned_df
cleaned_df2.head()

Dimension of cleaned_df:  (81142, 15)
Number of patients: 20403


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,charttime,text,diff_hosptime,diff_icutime,diff_charttime_icuintime,diff_charttime_icuoutime,diff_charttime_hospadmittime,diff_charttime_hospdischtime
0,24099382,2011-06-11 07:15:00,2011-06-18 21:45:00,2011-06-11 14:31:00,2011-06-13 18:05:00,2.15,0,2011-06-11 16:13:00,"INDICATION: Esophageal cancer, status post re...",7 days 14:30:00,2 days 03:34:00,0 days 01:42:00,-3 days +22:08:00,0 days 08:58:00,-8 days +18:28:00
1,24099382,2011-06-11 07:15:00,2011-06-18 21:45:00,2011-06-11 14:31:00,2011-06-13 18:05:00,2.15,0,2011-06-12 06:10:00,PORTABLE CHEST ___\n\nCOMPARISON: ___ radiogr...,7 days 14:30:00,2 days 03:34:00,0 days 15:39:00,-2 days +12:05:00,0 days 22:55:00,-7 days +08:25:00
2,27824879,2028-09-30 20:55:00,2028-10-10 15:54:00,2028-09-30 20:58:00,2028-10-05 18:39:00,4.90,0,2028-09-30 13:18:00,EXAMINATION: CT ABDOMEN AND PELVIS\n\nINDICAT...,9 days 18:59:00,4 days 21:41:00,-1 days +16:20:00,-6 days +18:39:00,-1 days +16:23:00,-11 days +21:24:00
3,27824879,2028-09-30 20:55:00,2028-10-10 15:54:00,2028-09-30 20:58:00,2028-10-05 18:39:00,4.90,0,2028-10-01 05:28:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,9 days 18:59:00,4 days 21:41:00,0 days 08:30:00,-5 days +10:49:00,0 days 08:33:00,-10 days +13:34:00
4,27146619,2041-09-20 00:34:00,2041-09-22 12:05:00,2041-09-20 07:13:00,2041-09-21 18:10:00,1.46,0,2041-09-20 01:01:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,2 days 11:31:00,1 days 10:57:00,-1 days +17:48:00,-2 days +06:51:00,0 days 00:27:00,-3 days +12:56:00


In [10]:
# subset the dataset with only required variables
data = cleaned_df2[['id','diff_charttime_icuintime','los_icu','icu_death','text']].copy()
data.head()

,id,diff_charttime_icuintime,los_icu,icu_death,text
0,24099382,0 days 01:42:00,2.15,0,"INDICATION: Esophageal cancer, status post re..."
1,24099382,0 days 15:39:00,2.15,0,PORTABLE CHEST ___\n\nCOMPARISON: ___ radiogr...
2,27824879,-1 days +16:20:00,4.90,0,EXAMINATION: CT ABDOMEN AND PELVIS\n\nINDICAT...
3,27824879,0 days 08:30:00,4.90,0,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...
4,27146619,-1 days +17:48:00,1.46,0,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...


In [11]:
# read the all text for ID: 24099382
example = data.groupby('id').get_group(24099382)

display(example)

print(example['text'].iloc[0])
print(example['text'].iloc[1])

,id,diff_charttime_icuintime,los_icu,icu_death,text
0,24099382,0 days 01:42:00,2.15,0,"INDICATION: Esophageal cancer, status post re..."
1,24099382,0 days 15:39:00,2.15,0,PORTABLE CHEST ___\n\nCOMPARISON: ___ radiogr...


INDICATION:  Esophageal cancer, status post resection, evaluation of the
nasogastric tube.

COMPARISON:  Preoperative chest x-ray from ___.

FINDINGS:  As compared to the previous radiograph, the patient has undergone
partial esophageal resection.  The right chest tube is in expected position. 
The tip of the nasogastric tube projects over the new esophagus, approximately
at the level of the right hilus.  There is no evidence of pneumothorax. 
Minimal postoperative atelectasis, minimal right soft tissue air collection.

Moderate cardiomegaly without pulmonary edema.  Unchanged position of the
right Port-A-Cath.

PORTABLE CHEST ___

COMPARISON:  ___ radiograph.

FINDINGS:  The patient is status post esophagectomy and pull-up procedure.
Nasogastric tube remains in place within the neoesophagus.  The neoesophagus
appears more distended with gas compared to the prior radiograph, accounting
for the widened right mediastinal contour on today's radiograph.  Right-sided
chest tube remains in p

In [12]:
# sort the 'diff_charttime_icutime' >>> give the sequence of the notes
sorted_df = data.groupby('id').apply(lambda x: x.sort_values('diff_charttime_icuintime', ascending=True))
sorted_df

/tmp/ipykernel_5095/4275520266.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = data.groupby('id').apply(lambda x: x.sort_values('diff_charttime_icuintime', ascending=True))


id diff_charttime_icuintime  los_icu  icu_death  \
id                                                                      
20001305 70611  20001305        -1 days +23:35:00     2.78          1   
         70612  20001305          0 days 16:22:00     2.78          1   
20001361 50268  20001361        -1 days +18:11:00     6.05          0   
         50269  20001361        -1 days +18:27:00     6.05          0   
         50270  20001361          0 days 01:33:00     6.05          0   
...                  ...                      ...      ...        ...   
29999625 29821  29999625        -1 days +21:46:00    22.13          0   
         29822  29999625        -1 days +22:31:00    22.13          0   
         29823  29999625          0 days 02:06:00    22.13          0   
         29824  29999625          0 days 02:18:00    22.13          0   
         29825  29999625          0 days 07:15:00    22.13          0   

                                                             text  
id                                                                 
20001305 70611  INDICATION:  ___ with copd in resp distress in...  
         70612  EXAMINATION:  UNILAT LOWER EXT VEINS RIGHT\n\n...  
20001361 50268  CHEST X-RAY DATED ___.\n\nCOMPARISON:  None.\n...  
         50269  CT OF THE HEAD DATED ___\n\nHISTORY: ___ yo ma...  
         50270  HISTORY:  Check tube placement.\n\nFINDINGS:  ...  
...                                                           ...  
29999625 29821  INDICATION:  ___ intubated// ?tube placement\n...  
         29822  EXAMINATION:  CT HEAD W/O CONTRAST\n\nINDICATI...  
         29823  EXAMINATION:  CT HEAD W/O CONTRAST\n\nINDICATI...  
         29824  EXAMINATION:  CHEST (PORTABLE AP)\n\nINDICATIO...  
         29825  EXAMINATION:  Chest x-ray\n\nINDICATION:  ___ ...  

[81142 rows x 5 columns]

In [13]:
# the number of records still greater than the number of patients
print('Dimension of sorted_df :', sorted_df.shape)

# reset the index for sorted_df
sorted_df2 = sorted_df.reset_index(drop=True)

# combined the notes for each patient 
combined_text_df = sorted_df2.groupby('id')['text'].apply('\n'.join).reset_index()

# check dimension of the combined_text_df
print('Dimension of combined_text_df : ', combined_text_df.shape)

# check number of patients in the combined_text_df
print('Number of patients: ', combined_text_df.id.nunique())

# display the first 5 rows of the combined_text_df
combined_text_df.head()

Dimension of sorted_df : (81142, 5)
Dimension of combined_text_df :  (20403, 2)
Number of patients:  20403


,id,text
0,20001305,INDICATION: ___ with copd in resp distress in...
1,20001361,CHEST X-RAY DATED ___.\n\nCOMPARISON: None.\n...
2,20001770,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
3,20002506,EXAMINATION: CTA HEAD AND CTA NECK Q16 CT NEC...
4,20003425,ADDENDUM Findings were communicated to the EN...


In [14]:
# read the text for ID: 24099382
example2 = combined_text_df[combined_text_df['id']==24099382]

display(example2)

print(example2['text'].iloc[0])

,id,text
8374,24099382,"INDICATION: Esophageal cancer, status post re..."


INDICATION:  Esophageal cancer, status post resection, evaluation of the
nasogastric tube.

COMPARISON:  Preoperative chest x-ray from ___.

FINDINGS:  As compared to the previous radiograph, the patient has undergone
partial esophageal resection.  The right chest tube is in expected position. 
The tip of the nasogastric tube projects over the new esophagus, approximately
at the level of the right hilus.  There is no evidence of pneumothorax. 
Minimal postoperative atelectasis, minimal right soft tissue air collection.

Moderate cardiomegaly without pulmonary edema.  Unchanged position of the
right Port-A-Cath.

PORTABLE CHEST ___

COMPARISON:  ___ radiograph.

FINDINGS:  The patient is status post esophagectomy and pull-up procedure.
Nasogastric tube remains in place within the neoesophagus.  The neoesophagus
appears more distended with gas compared to the prior radiograph, accounting
for the widened right mediastinal contour on today's radiograph.  Right-sided
chest tube remains in p

In [15]:
combined_text_final = combined_text_df.merge(data[['id','los_icu','icu_death']], on='id', how='left')

# keeping only the first row of each unique ID, the rest are duplicates
combined_text_final = combined_text_final.drop_duplicates(subset='id', keep='first')

# display the final dataframe 
display(combined_text_final.head())

# check dimension of the final dataframe
print('Dimension of combined_text_final:', combined_text_final.shape)

# check number of patients in the final dataframe
print('Number of patients:', combined_text_final.id.nunique())

,id,text,los_icu,icu_death
0,20001305,INDICATION: ___ with copd in resp distress in...,2.78,1
2,20001361,CHEST X-RAY DATED ___.\n\nCOMPARISON: None.\n...,6.05,0
7,20001770,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,2.87,0
11,20002506,EXAMINATION: CTA HEAD AND CTA NECK Q16 CT NEC...,6.56,0
14,20003425,ADDENDUM Findings were communicated to the EN...,4.00,0


Dimension of combined_text_final: (20403, 4)
Number of patients: 20403


In [16]:
combined_text_final.to_csv('../data/notes_combined.csv', index=False)